# Imports

### Execute in terminal: echo 1 | sudo tee /proc/sys/vm/overcommit_memory

In [1]:
# !pip install duckdb

In [2]:
import matplotlib.pyplot as plt
import pyarrow.parquet as pq
import pyarrow.dataset as ds
import tensorflow as tf
import pyarrow as pa
import scipy.spatial 
import pandas as pd
import numpy as np
import scipy.fft
import duckdb
import keras
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
# from biosppy.signals import ecg
from scipy import signal
from pandas import DataFrame
from pandas import concat
from pathlib import Path
from scipy import stats
from dtw import *

2023-09-15 22:42:54.239017: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-15 22:42:54.266830: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-15 22:42:54.472844: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-15 22:42:54.474686: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-15 22:42:56.594962: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



# Variables

In [3]:
root_data_emg_path = Path(r'/media/s792288955/TOSHIBA EXT/Andrea/Kaggle/ClassifyGestures_EMG/')

ecg_fs = 125  # frequency in Hz
ecg_lowcut = 0.5  # low frequency in Hz
ecg_highcut = 60  # hight frequency in Hz
ecg_window_length = 20
ecg_polyorder = 5

emg_fs = 200  # frequency in Hz
emg_lowcut = 0.5  # low frequency in Hz
emg_highcut = 90  # hight frequency in Hz
emg_window_length = 20
emg_polyorder = 5

# Datasets

In [4]:
# EEG

# 10 seconds
df_data_eeg_ori_v1_01_tcp_ar = pd.read_parquet(r'/media/s792288955/TOSHIBA EXT/Andrea/Datasets/10_sec/TUEG_V1_01_tcp_ar.parquet')
df_data_eeg_ori_v1_02_tcp_le = pd.read_parquet(r'/media/s792288955/TOSHIBA EXT/Andrea/Datasets/10_sec/TUEG_V1_02_tcp_le.parquet')
df_data_eeg_ori_v1_03_tcp_ar_a = pd.read_parquet(r'/media/s792288955/TOSHIBA EXT/Andrea/Datasets/10_sec/TUEG_V1_03_tcp_ar_a.parquet')
df_data_eeg_ori_v1_04_tcp_le_a = pd.read_parquet(r'/media/s792288955/TOSHIBA EXT/Andrea/Datasets/10_sec/TUEG_V1_04_tcp_le_a.parquet')

df_data_eeg_ori_v2_01_tcp_ar = pd.read_parquet(r'/media/s792288955/TOSHIBA EXT/Andrea/Datasets/10_sec/TUEG_V2_01_tcp_ar.parquet')
df_data_eeg_ori_v2_03_tcp_ar_a = pd.read_parquet(r'/media/s792288955/TOSHIBA EXT/Andrea/Datasets/10_sec/TUEG_V2_03_tcp_ar_a.parquet')

df_data_eeg_ori = pd.concat([df_data_eeg_ori_v1_01_tcp_ar, df_data_eeg_ori_v1_02_tcp_le,
                             df_data_eeg_ori_v1_03_tcp_ar_a, df_data_eeg_ori_v1_04_tcp_le_a,
                             df_data_eeg_ori_v2_01_tcp_ar, df_data_eeg_ori_v2_03_tcp_ar_a])

print(df_data_eeg_ori.shape)

con = duckdb.connect()
con.register('TUEG_EEG', df_data_eeg_ori)                     
# df_data_eeg_ori = con.execute(f"SELECT * FROM TUEG_EEG LIMIT {int(df_data_eeg_ori.shape[0]*0.1)};").df()
df_data_eeg_ori = con.execute(f"SELECT * FROM TUEG_EEG LIMIT {int(df_data_eeg_ori.shape[0]*0.08)};").df()

print(df_data_eeg_ori.shape)

(664477, 61)
(53158, 61)


In [5]:
#ECG
df_data_ecg_train = pd.read_csv(r'/media/s792288955/TOSHIBA EXT/Andrea/Kaggle/ECG-Dataset/mitbih_train.csv',
                          sep=',', header=None)

df_data_ecg_abnormal = pd.read_csv(r'/media/s792288955/TOSHIBA EXT/Andrea/Kaggle/ECG-Dataset/ptbdb_abnormal.csv',
                          sep=',', header=None)

df_data_ecg_normal = pd.read_csv(r'/media/s792288955/TOSHIBA EXT/Andrea/Kaggle/ECG-Dataset/ptbdb_normal.csv',
                          sep=',', header=None)


df_data_ecg_train_ori = pd.concat([df_data_ecg_train, df_data_ecg_abnormal, df_data_ecg_normal], axis=0)

print(df_data_ecg_train_ori.shape)

con = duckdb.connect()
con.register('ECG', df_data_ecg_train_ori)                    
df_data_ecg_train_ori = con.execute(f"SELECT * FROM ECG LIMIT {int(df_data_ecg_train_ori.shape[0]*0.08)};").df()

print(df_data_ecg_train_ori.shape)

(102106, 188)
(8168, 188)


In [6]:
#EMG
all_filenames = [i for i in root_data_emg_path.glob('*.{}'.format('csv'))]

df_data_emg_0_ori = pd.read_csv(all_filenames[0], 
                                sep=",", header=None)

df_data_emg_1_ori = pd.read_csv(all_filenames[1], 
                                sep=",", header=None)

df_data_emg_2_ori = pd.read_csv(all_filenames[2], 
                                 sep=",", header=None)

df_data_emg_3_ori = pd.read_csv(all_filenames[3], 
                                   sep=",", header=None)

df_data_emg_ori   = pd.concat([df_data_emg_0_ori,df_data_emg_1_ori,df_data_emg_2_ori,df_data_emg_3_ori], axis=0)

print(df_data_emg_ori.shape)

(11678, 65)


# Functions

# Pre Processing

### Steps EEG

#### 1 - Selec groups of signals 

In [7]:
# Split into X and Y
X_train_eeg = df_data_eeg_ori[["A2", "F3", "F4", "F7", "F8", "FP1", "FP2", "FZ", 
                               "T3", "T4", "T5", "T6", "C3", "C4", "CZ", "O1", 
                               "O2", "P3", "P4", "PZ"]].copy()

print(X_train_eeg.shape)

(53158, 20)


In [8]:
# ensure all data is float
X_train_eeg = X_train_eeg.astype('float64')

# The input dims are 20
X_train_eeg_np = X_train_eeg.to_numpy()

# Normalizing the amplitude values to the range of between zero and one.
scaler_eeg = MinMaxScaler()

X_train_eeg_filtered = X_train_eeg_np.reshape(X_train_eeg_np.shape[0], -1)
X_train_eeg_np_normalized = scaler_eeg.fit_transform(X_train_eeg_filtered)

# Add a channel dimension to the input
X_train_eeg_np = X_train_eeg_np_normalized.reshape(-1, 20)

# X_train_eeg_np = X_train_eeg_np.reshape(-1, 20)

print("X_train_eeg_np = ", X_train_eeg_np.shape)

X_train_eeg_np =  (53158, 20)


In [9]:
# Fourier Transforms (FFTs) to work in frequency domain.
df_train_eeg_freq  = scipy.fft.fft2(X_train_eeg_np).view(np.float64)

print(df_train_eeg_freq.shape)

(53158, 40)


### Steps ECG

#### 1 - Segment ECG signals in groups of 29 features.

In [10]:
# split into input 
X_train_ecg = df_data_ecg_train_ori.iloc[:,   :-2].copy()

print(X_train_ecg.shape)

(8168, 186)


In [11]:
# ensure all data is float
X_train_ecg = X_train_ecg.astype('float64')

# The input has to be (batch_size, 1, 187)
X_train_ecg_np = X_train_ecg.to_numpy()

# Normalizing the amplitude values to the range of between zero and one.
scaler_ecg = MinMaxScaler()

X_train_ecg_np = X_train_ecg_np.reshape(X_train_ecg_np.shape[0], -1)
X_train_ecg_np_normalized = scaler_ecg.fit_transform(X_train_ecg_np)

# Add a channel dimension to the input
X_train_ecg_np = X_train_ecg_np_normalized.reshape(-1, 186)

# X_train_ecg_np = X_train_ecg_np.reshape(-1, 186)

print("X_train_ecg_np = ", X_train_ecg_np.shape)

X_train_ecg_np =  (8168, 186)


In [12]:
X_train_ecg_1 = X_train_ecg_np[:,    :20].copy()
X_train_ecg_2 = X_train_ecg_np[:,  20:40].copy()
X_train_ecg_3 = X_train_ecg_np[:,  40:60].copy()
X_train_ecg_4 = X_train_ecg_np[:,  60:80].copy()
X_train_ecg_5 = X_train_ecg_np[:,  80:100].copy()
X_train_ecg_6 = X_train_ecg_np[:, 100:120].copy()
X_train_ecg_7 = X_train_ecg_np[:, 120:140].copy()
X_train_ecg_8 = X_train_ecg_np[:, 140:160].copy()
X_train_ecg_9 = X_train_ecg_np[:, 160:180].copy()

print(X_train_ecg_1.shape)
print(X_train_ecg_2.shape)
print(X_train_ecg_3.shape)
print(X_train_ecg_4.shape)
print(X_train_ecg_5.shape)
print(X_train_ecg_6.shape)
print(X_train_ecg_7.shape)
print(X_train_ecg_8.shape)
print(X_train_ecg_9.shape)

(8168, 20)
(8168, 20)
(8168, 20)
(8168, 20)
(8168, 20)
(8168, 20)
(8168, 20)
(8168, 20)
(8168, 20)


In [13]:
# remove low and high noise frequecy
ecg_b, ecg_a = signal.butter(4, [ecg_lowcut / (ecg_fs/2), ecg_highcut / (ecg_fs/2)], 'band')
X_train_ecg_1_filtered = signal.filtfilt(ecg_b, ecg_a, X_train_ecg_1, padlen=X_train_ecg_1.shape[1]-1)
X_train_ecg_2_filtered = signal.filtfilt(ecg_b, ecg_a, X_train_ecg_2, padlen=X_train_ecg_2.shape[1]-1)
X_train_ecg_3_filtered = signal.filtfilt(ecg_b, ecg_a, X_train_ecg_3, padlen=X_train_ecg_3.shape[1]-1)
X_train_ecg_4_filtered = signal.filtfilt(ecg_b, ecg_a, X_train_ecg_4, padlen=X_train_ecg_4.shape[1]-1)
X_train_ecg_5_filtered = signal.filtfilt(ecg_b, ecg_a, X_train_ecg_5, padlen=X_train_ecg_5.shape[1]-1)
X_train_ecg_6_filtered = signal.filtfilt(ecg_b, ecg_a, X_train_ecg_6, padlen=X_train_ecg_6.shape[1]-1)
X_train_ecg_7_filtered = signal.filtfilt(ecg_b, ecg_a, X_train_ecg_7, padlen=X_train_ecg_7.shape[1]-1)
X_train_ecg_8_filtered = signal.filtfilt(ecg_b, ecg_a, X_train_ecg_8, padlen=X_train_ecg_8.shape[1]-1)
X_train_ecg_9_filtered = signal.filtfilt(ecg_b, ecg_a, X_train_ecg_9, padlen=X_train_ecg_9.shape[1]-1)


# Apply moving average to remove high noise
X_train_ecg_1_filtered = signal.savgol_filter(X_train_ecg_1_filtered, ecg_window_length, ecg_polyorder)
X_train_ecg_2_filtered = signal.savgol_filter(X_train_ecg_2_filtered, ecg_window_length, ecg_polyorder)
X_train_ecg_3_filtered = signal.savgol_filter(X_train_ecg_3_filtered, ecg_window_length, ecg_polyorder)
X_train_ecg_4_filtered = signal.savgol_filter(X_train_ecg_4_filtered, ecg_window_length, ecg_polyorder)
X_train_ecg_5_filtered = signal.savgol_filter(X_train_ecg_5_filtered, ecg_window_length, ecg_polyorder)
X_train_ecg_6_filtered = signal.savgol_filter(X_train_ecg_6_filtered, ecg_window_length, ecg_polyorder)
X_train_ecg_7_filtered = signal.savgol_filter(X_train_ecg_7_filtered, ecg_window_length, ecg_polyorder)
X_train_ecg_8_filtered = signal.savgol_filter(X_train_ecg_8_filtered, ecg_window_length, ecg_polyorder)
X_train_ecg_9_filtered = signal.savgol_filter(X_train_ecg_9_filtered, ecg_window_length, ecg_polyorder)


In [14]:
# Fourier Transforms (FFTs) to work in frequency domain.
df_ecg_1_freq  = scipy.fft.fft2(X_train_ecg_1_filtered).view(np.float64)
df_ecg_2_freq  = scipy.fft.fft2(X_train_ecg_2_filtered).view(np.float64)
df_ecg_3_freq  = scipy.fft.fft2(X_train_ecg_3_filtered).view(np.float64)
df_ecg_4_freq  = scipy.fft.fft2(X_train_ecg_4_filtered).view(np.float64)
df_ecg_5_freq  = scipy.fft.fft2(X_train_ecg_5_filtered).view(np.float64)
df_ecg_6_freq  = scipy.fft.fft2(X_train_ecg_6_filtered).view(np.float64)
df_ecg_7_freq  = scipy.fft.fft2(X_train_ecg_7_filtered).view(np.float64)
df_ecg_8_freq  = scipy.fft.fft2(X_train_ecg_8_filtered).view(np.float64)
df_ecg_9_freq  = scipy.fft.fft2(X_train_ecg_9_filtered).view(np.float64)


In [15]:
print(df_ecg_1_freq.shape)
print(df_ecg_2_freq.shape)
print(df_ecg_3_freq.shape)
print(df_ecg_4_freq.shape)
print(df_ecg_5_freq.shape)
print(df_ecg_6_freq.shape)
print(df_ecg_7_freq.shape)
print(df_ecg_8_freq.shape)
print(df_ecg_9_freq.shape)

(8168, 40)
(8168, 40)
(8168, 40)
(8168, 40)
(8168, 40)
(8168, 40)
(8168, 40)
(8168, 40)
(8168, 40)


### Steps EMG

#### 1 - Divide the EMG in 4 groups (repects the extraction strategy)

In [16]:
# Split into X
X_train_emg = df_data_emg_ori.iloc[:,   :-1].copy()

print(X_train_emg.shape)

(11678, 64)


In [17]:
# ensure all data is float
X_train_emg = X_train_emg.astype('float64')

# The input dims are 64 
X_train_emg_np = X_train_emg.to_numpy()

# Normalizing the amplitude values to the range of between zero and one.
scaler_emg = MinMaxScaler()

X_train_emg_np = X_train_emg_np.reshape(X_train_emg_np.shape[0], -1)
X_train_emg_np_normalized = scaler_emg.fit_transform(X_train_emg_np)

# Add a channel dimension to the input
X_train_emg_np = X_train_emg_np_normalized.reshape(-1, 64)

# X_train_emg_np = X_train_emg_np.reshape(-1, 64)

print("X_train_emg_np = ", X_train_emg_np.shape)

X_train_emg_np =  (11678, 64)


In [18]:
X_train_emg_1 = X_train_ecg_np[:,    :20].copy()
X_train_emg_2 = X_train_ecg_np[:,  20:40].copy()
X_train_emg_3 = X_train_ecg_np[:,  40:60].copy()

print(X_train_emg_1.shape)
print(X_train_emg_2.shape)
print(X_train_emg_3.shape)

(8168, 20)
(8168, 20)
(8168, 20)


In [19]:
# signal.butter - Design an Nth-order digital or analog Butterworth filter and return the filter coefficients.
emg_b, emg_a = signal.butter(4, [emg_lowcut / (emg_fs/2), emg_highcut / (emg_fs/2)], 'band')
X_train_emg_1_filtered = signal.filtfilt(emg_b, emg_a, X_train_emg_1, padlen=X_train_emg_1.shape[1]-1)
X_train_emg_2_filtered = signal.filtfilt(emg_b, emg_a, X_train_emg_2, padlen=X_train_emg_2.shape[1]-1)
X_train_emg_3_filtered = signal.filtfilt(emg_b, emg_a, X_train_emg_3, padlen=X_train_emg_3.shape[1]-1)

# Apply moving average to remove high noise
X_train_emg_1_filtered = signal.savgol_filter(X_train_emg_1_filtered, emg_window_length, emg_polyorder)
X_train_emg_2_filtered = signal.savgol_filter(X_train_emg_2_filtered, emg_window_length, emg_polyorder)
X_train_emg_3_filtered = signal.savgol_filter(X_train_emg_3_filtered, emg_window_length, emg_polyorder)

In [20]:
# Fourier Transforms (FFTs) to work in frequency domain.
df_emg_1_freq  = scipy.fft.fft2(X_train_emg_1_filtered).view(np.float64)
df_emg_2_freq  = scipy.fft.fft2(X_train_emg_2_filtered).view(np.float64)
df_emg_3_freq  = scipy.fft.fft2(X_train_emg_3_filtered).view(np.float64)


In [21]:
print(df_emg_1_freq.shape)
print(df_emg_2_freq.shape)
print(df_emg_3_freq.shape)

(8168, 40)
(8168, 40)
(8168, 40)


# Similarity evaluation

## Steps

### 1 -  Calculate the dtw's similarity rate for each group
### 2 - Select the group with lower rate to train the model and transfer the learning

### ECG-EEG Similarities

In [22]:
dtw_ecg_eeg_frontal_1  = dtw(df_train_eeg_freq, df_ecg_1_freq)
dtw_ecg_eeg_frontal_2  = dtw(df_train_eeg_freq, df_ecg_2_freq)
dtw_ecg_eeg_frontal_3  = dtw(df_train_eeg_freq, df_ecg_3_freq)
dtw_ecg_eeg_frontal_4  = dtw(df_train_eeg_freq, df_ecg_4_freq)
dtw_ecg_eeg_frontal_5  = dtw(df_train_eeg_freq, df_ecg_5_freq)
dtw_ecg_eeg_frontal_6  = dtw(df_train_eeg_freq, df_ecg_6_freq)
dtw_ecg_eeg_frontal_7  = dtw(df_train_eeg_freq, df_ecg_7_freq)
dtw_ecg_eeg_frontal_8  = dtw(df_train_eeg_freq, df_ecg_8_freq)
dtw_ecg_eeg_frontal_9  = dtw(df_train_eeg_freq, df_ecg_9_freq)

In [23]:
print(dtw_ecg_eeg_frontal_1.distance)
print(dtw_ecg_eeg_frontal_2.distance)
print(dtw_ecg_eeg_frontal_3.distance)
print(dtw_ecg_eeg_frontal_4.distance)
print(dtw_ecg_eeg_frontal_5.distance)
print(dtw_ecg_eeg_frontal_6.distance)
print(dtw_ecg_eeg_frontal_7.distance)
print(dtw_ecg_eeg_frontal_8.distance)
print(dtw_ecg_eeg_frontal_9.distance)

15760857.804772621
13472402.478133228
13680336.216750799
15381831.09107019
16991571.016822305
15334382.71187192
14907344.557979504
13825110.35204897
13059530.498655513


### EEG-EMG Similarities

In [24]:
#EMG group 1
dtw_ecg_emg_group1_1  = dtw(df_emg_1_freq, df_ecg_1_freq)
dtw_ecg_emg_group1_2  = dtw(df_emg_1_freq, df_ecg_2_freq)
dtw_ecg_emg_group1_3  = dtw(df_emg_1_freq, df_ecg_3_freq)
dtw_ecg_emg_group1_4  = dtw(df_emg_1_freq, df_ecg_4_freq)
dtw_ecg_emg_group1_5  = dtw(df_emg_1_freq, df_ecg_5_freq)
dtw_ecg_emg_group1_6  = dtw(df_emg_1_freq, df_ecg_6_freq)
dtw_ecg_emg_group1_7  = dtw(df_emg_1_freq, df_ecg_7_freq)
dtw_ecg_emg_group1_8  = dtw(df_emg_1_freq, df_ecg_8_freq)
dtw_ecg_emg_group1_9  = dtw(df_emg_1_freq, df_ecg_9_freq)

In [25]:
# ECG-EMG Group1

print(dtw_ecg_emg_group1_1.distance)
print(dtw_ecg_emg_group1_2.distance)
print(dtw_ecg_emg_group1_3.distance)
print(dtw_ecg_emg_group1_4.distance)
print(dtw_ecg_emg_group1_5.distance)
print(dtw_ecg_emg_group1_6.distance)
print(dtw_ecg_emg_group1_7.distance)
print(dtw_ecg_emg_group1_8.distance)
print(dtw_ecg_emg_group1_9.distance)

1340306.2472125462
3553066.498230459
3645908.8747261507
4707653.567468465
5240276.331126137
4494367.626354662
4136901.654236917
3621572.598349525
3255046.6210161946


In [26]:
#EMG group 2
dtw_ecg_emg_group2_1  = dtw(df_emg_2_freq, df_ecg_1_freq)
dtw_ecg_emg_group2_2  = dtw(df_emg_2_freq, df_ecg_2_freq)
dtw_ecg_emg_group2_3  = dtw(df_emg_2_freq, df_ecg_3_freq)
dtw_ecg_emg_group2_4  = dtw(df_emg_2_freq, df_ecg_4_freq)
dtw_ecg_emg_group2_5  = dtw(df_emg_2_freq, df_ecg_5_freq)
dtw_ecg_emg_group2_6  = dtw(df_emg_2_freq, df_ecg_6_freq)
dtw_ecg_emg_group2_7  = dtw(df_emg_2_freq, df_ecg_7_freq)
dtw_ecg_emg_group2_8  = dtw(df_emg_2_freq, df_ecg_8_freq)
dtw_ecg_emg_group2_9  = dtw(df_emg_2_freq, df_ecg_9_freq)

In [27]:
# ECG-EMG Group2

print(dtw_ecg_emg_group2_1.distance)
print(dtw_ecg_emg_group2_2.distance)
print(dtw_ecg_emg_group2_3.distance)
print(dtw_ecg_emg_group2_4.distance)
print(dtw_ecg_emg_group2_5.distance)
print(dtw_ecg_emg_group2_6.distance)
print(dtw_ecg_emg_group2_7.distance)
print(dtw_ecg_emg_group2_8.distance)
print(dtw_ecg_emg_group2_9.distance)



4093548.9832320595
266810.0151487971
2017561.8501622311
3430521.7335955994
4216798.181652939
3369093.46295432
2920498.624273856
2141149.392583915
1593389.1851932856


In [28]:
#EMG group 3
dtw_ecg_emg_group3_1  = dtw(df_emg_3_freq, df_ecg_1_freq)
dtw_ecg_emg_group3_2  = dtw(df_emg_3_freq, df_ecg_2_freq)
dtw_ecg_emg_group3_3  = dtw(df_emg_3_freq, df_ecg_3_freq)
dtw_ecg_emg_group3_4  = dtw(df_emg_3_freq, df_ecg_4_freq)
dtw_ecg_emg_group3_5  = dtw(df_emg_3_freq, df_ecg_5_freq)
dtw_ecg_emg_group3_6  = dtw(df_emg_3_freq, df_ecg_6_freq)
dtw_ecg_emg_group3_7  = dtw(df_emg_3_freq, df_ecg_7_freq)
dtw_ecg_emg_group3_8  = dtw(df_emg_3_freq, df_ecg_8_freq)
dtw_ecg_emg_group3_9  = dtw(df_emg_3_freq, df_ecg_9_freq)

In [29]:
# ECG-EMG Group3

print(dtw_ecg_emg_group3_1.distance)
print(dtw_ecg_emg_group3_2.distance)
print(dtw_ecg_emg_group3_3.distance)
print(dtw_ecg_emg_group3_4.distance)
print(dtw_ecg_emg_group3_5.distance)
print(dtw_ecg_emg_group3_6.distance)
print(dtw_ecg_emg_group3_7.distance)
print(dtw_ecg_emg_group3_8.distance)
print(dtw_ecg_emg_group3_9.distance)


4181646.8972145277
2005082.7887081318
315080.0363121554
3504677.2269330733
4280958.175688838
3438431.1960294074
3006543.092549872
2267203.069618775
1732921.4092676388
